In [7]:
import pandas as pd
import geopandas as gpd
import folium
import geopy
from geopy.distance import geodesic
from shapely.geometry import Point
import branca.colormap as cm
from ipywidgets import interact_manual, widgets, VBox, HBox
import ipywidgets as widgets
from sklearn.preprocessing import MinMaxScaler
import zipfile
import os
import fiona
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
pois_linestrings= gpd.read_file("pois_linestrings.shp")
pois_points= gpd.read_file("pois_points.shp")
pois_poligonos= gpd.read_file("pois_poligonos.shp")

In [9]:
# pois_linestrings
pois_poligonos[pois_poligonos['fclass']=='police']

,osm_id,fclass,name,numero_df,type,geometry
294,93249276,police,None,0,None,"POLYGON ((-46.53407 -23.57987, -46.53356 -23.5..."
321,104291524,police,Polícia Militar - Comando de Policiamento da C...,0,None,"POLYGON ((-46.63421 -23.53194, -46.63421 -23.5..."
434,121546222,police,99º DP - Campo Grande,0,None,"POLYGON ((-46.68728 -23.66211, -46.68720 -23.6..."
912,2070362,police,Academia de Polícia - Campus I,0,None,"POLYGON ((-46.71329 -23.56579, -46.71327 -23.5..."
1007,155852282,police,75º Distrito Policial (Jardim Arpoador),0,None,"POLYGON ((-46.79625 -23.59726, -46.79569 -23.5..."
1008,155859382,police,Polícia Militar Rodoviária,0,None,"POLYGON ((-46.79183 -23.58704, -46.79132 -23.5..."
1076,158967598,police,16º Batalhão da Polícia Militar,0,None,"POLYGON ((-46.74728 -23.55913, -46.74718 -23.5..."
1077,158967600,police,93º Distrito Policial,0,None,"POLYGON ((-46.74684 -23.55879, -46.74681 -23.5..."
1078,159112172,police,35º Distrito Policial,0,None,"POLYGON ((-46.64350 -23.63933, -46.64337 -23.6..."
1097,160689310,police,Polícia Federal Superintendência Regional de S...,0,None,"POLYGON ((-46.70500 -23.51095, -46.70490 -23.5..."


# Funções de Estações

In [4]:
## Prepara o dataframe de estações para as outras análises
## Retorna o dataframe tratado
def tratar_df_estacoes(df_estacoes):
    ## Cria uma geometria "Point"
    df_estacoes = gpd.GeoDataFrame(df_estacoes, geometry = gpd.points_from_xy(df_estacoes['lon'], df_estacoes['lat']))
    ## Define crs
    df_estacoes.set_crs("EPSG:4326", inplace = True)
    return df_estacoes

## Cria circles (geometry Polygon) em volta de cada estação (geometry Point)
## Retorna o dataframe com a geometria "circles"
def criar_circulos_estacao(data, radius_km):
    circles = []
    for _, row in data.iterrows():
        circle_points = []
        center = (row['lat'], row['lon'])
        ## Dizemos que é aproximadamente um círculo, pois é um polígono regular com 360 vértices
        for angle in range(0,360, 1): 
            destination = geopy.distance.distance(kilometers = radius_km).destination(center, angle)
            circle_points.append(Point(destination.longitude, destination.latitude))
        circle = gpd.GeoSeries(circle_points).unary_union.convex_hull
        circles.append(circle)
    data.rename(columns = {'geometry':'point'}, inplace = True)
    circles_gdf = gpd.GeoDataFrame(data, geometry = circles)
    circles_gdf.rename(columns = {'geometry':'circle'}, inplace = True)
    return circles_gdf

## Adiciona os círculos no mapa, usa como base a função "criar_circulos_estacao"
## Retorna o Mapa com os círculos (geometry polygon) e pontos (geometry point) adicionados
def adicionar_estacoes_com_circulos_mapa(mapa, df_estacoes, radius_km):
    ## Definição dos layers para controle do usuário por camadas
    layer_estacoes =  folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 
    ## Se o df_estacoes já tem os circles, não precisa criá-los
    if "circle" not in df_estacoes:
        data = criar_circulos_estacao(df_estacoes, radius_km)
    else:
        data = df_estacoes.copy()
    ## Define "circle" como geometria a ser usada, pois há mais de uma em uso
    data.set_geometry("circle", inplace = True)
    ## Adiciona Circlemarkers ao layer
    for _, row in data.iterrows():
        folium.CircleMarker(
                    location=[row['lat'],row['lon']],
                    color = "blue",
                    radius= 5,
                    tooltip= row['name'],
                    fill=True,
                    fill_opacity=1,
                    fill_color="blue",
                ).add_to(layer_estacoes)
    ## Adiciona os Circles ao layer
    for _, row in data.iterrows():
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        layer_estacoes.add_to(mapa)
        layer_circulos_estacoes.add_to(mapa)
        folium.LayerControl().add_to(mapa)
    return mapa

# Funções de Mapa

In [5]:
## Define cores para mapas
def get_color(valor):
    if valor == 1:
        return "yellow"
    elif valor == 2:
        return "orange"
    elif valor == 3:
        return "red"
    elif valor == 4:
        return "darkred"
    elif valor == 5:
        return "blue"
    elif valor == 6:
        return "darkblue"
    elif valor == 7:
        return "purple"
    elif valor == 8:
        return "pink"
    elif valor == 9:
        return "green"
    elif valor == 10:
        return "darkgreen"
    elif valor == 11:
        return "lightblue"
    elif valor == 12:
        return "cyan"
    elif valor == 13:
        return "brown"
    elif valor == 14:
        return "gray"
    elif valor == 15:
        return "black"
    elif valor == 16:
        return "lightgray"
    elif valor == 17:
        return "white"
    else:
        return "black"  # Valor inválido

## Cria um mapa centrado em São Paulo
def criar_mapa_sp():
    mapa_sp = folium.Map(location = [-23.550520, -46.633308], zoom_start = 12)
    return mapa_sp

## Plota Circlemarkers de 1 cor, dado 1 df com Lat e Lon
def plot_circlemarkers_one_color(data, color, radius, name, pontuacao, layer):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = color, 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup = row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = color,
        ).add_to(layer)

## Plota Circlemarkers de várias cores, dado 1 df com Lat e Lon e um Colormap branca
def plot_circlemarkers_multi_color(data, radius, name, pontuacao, layer, colormap):
    for _, row in data.iterrows():
        folium.CircleMarker(
                location = [row['lat'], row['lon']],
                color = colormap(row[pontuacao]), 
                radius = radius,
                tooltip = row[name] + " " + str(row[pontuacao]), 
                popup =  row[name] + " " + str(row[pontuacao]), 
                fill = True,
                fill_opacity = 1,
                fill_color = colormap(row[pontuacao]),
        ).add_to(layer)
        
    


## Funções de Outliers

In [6]:
## Plota os Circlemarkers outliers dado um out_lower, out_upper e non_outliers, separados pela função "identificar_outliers"
def plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, color_lower, color_upper, radius, 
                                name, pontuacao, layer, colormap):
    plot_circlemarkers_one_color(out_lower, color_lower, radius, name, pontuacao, layer)
    plot_circlemarkers_one_color(out_upper, color_upper, radius, name, pontuacao, layer)
    plot_circlemarkers_multi_color(non_outliers, radius, name, pontuacao, layer, colormap)

def identificar_outliers(df, coluna):
    """
    Identifica outliers em uma coluna de um DataFrame usando o método IQR.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        coluna (str): Nome da coluna para identificar os outliers.

    Retorna:
        outliers_lower (pd.DataFrame): DataFrame contendo os outliers inferiores.
        outliers_upper (pd.DataFrame): DataFrame contendo os outliers superiores.
        non_outliers (pd.DataFrame): DataFrame contendo os dados não-outliers.
    """
    Q1 = df[coluna].quantile(0.25) 
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    lowerbound = Q1 - 1.5 * IQR
    upperbound = Q3 + 1.5 * IQR

    outliers_lower = df[df[coluna] < lowerbound]
    outliers_upper = df[df[coluna] > upperbound]
    non_outliers = df[(df[coluna] >= lowerbound) & (df[coluna] <= upperbound)]

    return outliers_lower, outliers_upper, non_outliers 


# Funções de Uso do Solo

In [62]:
## Prepara o dataframe de ocorrencias para as outras análises
## Retorna o dataframe tratado
def tratar_df_pois(df_pois):
    pois = df_pois.copy()
    pois = pois.rename(columns={'name':'poi_name'})
    pois = pois.to_crs("EPSG:4326")
    return pois
    
def analise_pois(df_pois, df_estacoes, radius_km, pois_selecionados):
    mapa = criar_mapa_sp()
    df_estacoes_circulo = criar_circulos_estacao(df_estacoes, radius_km)
    df_estacoes_circulo.set_geometry("circle", inplace=True)  

    df_estacoes_circulo.set_crs("EPSG:4326", inplace = True)
    ## Quais Pois Considerar?
    df_pois_filtrado = df_pois[df_pois['fclass'].isin(pois_selecionados)]
    
    cruzamentos =  gpd.sjoin(df_estacoes_circulo, df_pois_filtrado, how="inner", predicate="intersects")
    
    cruzamentos = cruzamentos.merge(
        df_pois[["osm_id", "geometry"]],  # Apenas as colunas relevantes
        on="osm_id",  # A coluna para equivalência
        how="left"  # Tipo de junção (left mantém todos os dados de cruzamentos)
    )
    ocorrencias_necessarias_para_plot = cruzamentos.copy()
  
    layer_estacoes = folium.FeatureGroup(name = "Estações")
    layer_circulos_estacoes = folium.FeatureGroup(name = f"Redondezas {radius_km} km") 

    ## Não vou plotar pois, melhor não, tem mts
    # ## Plot do uso do solo
    # layers_dict = {}
    # for place in pois_selecionados:
    #     ocorrencia_por_plot = ocorrencias_necessarias_para_plot[ocorrencias_necessarias_para_plot['fclass'] == place]
    #     layer_pois = folium.FeatureGroup(
    #     name=str(ocorrencias_necessarias_para_plot['pois_selecionados']
    #          [ocorrencias_necessarias_para_plot['fclass'] == place].values[0]),
    #     overlay = True) 
    #     layers_dict[place] = layer_pois
    #     for _, row in ocorrencia_por_plot.iterrows():
    #         cor = get_color(row['fclass'])
    #         geo_json = row['geometry'].__geo_interface__
    #         folium.GeoJson(
    #             geo_json, 
    #             tooltip = str(row['Name']) + " Classe " + str(row['pois_selecionados']),
    #             style_function = lambda x, color = cor:{'fillColor' : color , 'color' : color, 'weight' : 1}
    #         ).add_to(layers_dict[place])
    #     layers_dict[place].add_to(mapa)
        # layers_dict[place].control = False
    # folium.LayerControl(collapsed=False).add_to(mapa)
    # return mapa

    cruzamentos = cruzamentos.groupby('name', as_index = False)['fclass'].count()
    cruzamentos= pd.merge(df_estacoes_circulo, cruzamentos, on = 'name',  how = 'left').fillna(0)
    # return cruzamentos
    # return cruzamentos.isnull().sum()

    # scaler = MinMaxScaler()
    # cruzamentos['fclass'] = scaler.fit_transform(cruzamentos[['fclass']])
    out_lower, out_upper, non_outliers = identificar_outliers(cruzamentos, "fclass")
    colormap = cm.linear.plasma.scale(non_outliers['fclass'].min(), 
                                      non_outliers['fclass'].max())
    colormap.add_to(mapa)
    
    plot_circlemarkers_outliers(out_lower, out_upper, non_outliers, "black", "green", 5, 
                                "name", "fclass", layer_estacoes, colormap)  

    ## Plota círculos em volta das estações
    for _, row in df_estacoes_circulo.iterrows():    
        folium.GeoJson(
            row['circle'].__geo_interface__, 
            tooltip = row['name'],
            style_function=lambda x:{'fillColor': 'gray', 'color': 'gray', 'fillOpacity': 0.0},
        ).add_to(layer_circulos_estacoes)
        
    # layer_pois.add_to(mapa)
    layer_estacoes.add_to(mapa)
    layer_circulos_estacoes.add_to(mapa)
    folium.LayerControl().add_to(mapa)
    return mapa


def roda_analise_pois(geodata, df_estacoes):
    im = interact_manual(
        lambda pois_selecionados, radius_km: analise_pois(geodata,df_estacoes, radius_km, pois_selecionados) ,# Passa df_fixo sem interação
        pois_selecionados=widgets.SelectMultiple(options=[classe for classe in geodata['fclass'].unique()]),
        radius_km = widgets.FloatSlider(min=0, max=2, value=0.5),
    )
    im.widget.children[0].description = 'Quais Pois?'
    im.widget.children[1].description = 'Raio em KM'


In [63]:
df_pois_all_geometries = pd.concat([pois_points, pois_poligonos, pois_linestrings], ignore_index=True)

df_pois = tratar_df_pois(pois_points)
df_estacoes = tratar_df_estacoes(estacoes)

In [64]:
roda_analise_pois(df_pois, df_estacoes)

interactive(children=(SelectMultiple(description='pois_selecionados', options=('attraction', 'fast_food', 'mal…

In [13]:
estacoes = pd.read_parquet("estacoes_reestruturadas.parquet")

In [59]:
# pois_points
# df_estacoes

In [19]:
df_pois = tratar_df_pois(pois)
df_estacoes = tratar_df_estacoes(estacoes)

In [20]:
roda_analise_pois(pois_points, df_estacoes)

interactive(children=(Dropdown(description='pois_selecionados', options=('attraction', 'fast_food', 'mall', 'c…

In [ ]:
analise_pois(df_pois, df_estacoes, 1, )

In [ ]:
df_pois.columns

In [ ]:
df_pois.head()

In [ ]:
df_pois['fclass'].value_counts()

In [ ]:
# df_pois['uso_pred_60'].value_counts()
df_pois.shape